# Homework: b)
Simon Burkhardt, FHNW/ISE, 05.05.2021

b) Look at the following citcuit:

<img src="Schematic_GxGy.png" alt="sfg" width="400"/>

The goal is to calculate the current through Gy. Proceed as follows:

1. Calculate the voltage across $G_y$ for general $A$ using a DPSDG.
2. From that, calculate the current through $G_y$.
3. Simplify for $A → ∞$. What value do you have to choose for $G_x$ such that the current through $G_y$ does not depend on $G_y$ anymore?
4. For that choice of $G_x$ and $A \approx \frac{\omega_1}{s}$, what is the bandwidth of the circuit?

<img src="SFG_GxGy.png" alt="sfg" width="600"/>


In [9]:
%matplotlib notebook
import sympy as sp
import numpy as np
import math
import scipy as sc
import scipy.signal
import matplotlib.pyplot as plt
import hanspitools as ht
def niceT(T,s): # This is a shortcut for a functionality I use very often
    return ht.mani.numden(lambda p: ht.mani.nicepoly(p,s),T)
def keeper(f,v):
    return sp.limit(f/v,v,sp.oo)*v

In [16]:
import sympy as sp
Delta = sp.symbols('Delta')
L1,L2 = sp.symbols('L1,L2')
T_num = sp.symbols('T_num')
T_den = sp.symbols('T_den')
T_io = sp.symbols('T_io')
Z2,A,G,Z4,Gx = sp.symbols('Z2,A,G,Z4,Gx')

loops = [(L1, -A*G*Z4), (L2, A*G*Z2)]
determinant = [(Delta, -L1 - L2 + 1)]
denominator = [(T_den, Delta)]

P1,D1 = sp.symbols('P1,D1')
paths = [(P1, Gx*Z2), (D1, 1 - L1)]
numerator = [(T_num, D1*P1)]

transfer_function = [(T_io, T_num/T_den)]
T=T_io.subs(transfer_function).subs(numerator).subs(denominator).subs(determinant).subs(paths).subs(loops).simplify()
display(T)

Gx*Z2*(A*G*Z4 + 1)/(-A*G*Z2 + A*G*Z4 + 1)

In [23]:
Y2,Y4,Vin=sp.symbols('Y2,Y4,Vin')
Gy,s,omega1 = sp.symbols('Gy,s,omega1')
eqY2 = [(Y2, (Gx+Gy+G))] # is this correct +/- ?
eqY4 = [(Y4, (G+G))]
eqZ2 = [(Z2, 1/Y2)]
eqZ4 = [(Z4, 1/Y4)]
amp = [(A,  omega1/s)]
T_w = T.subs(eqZ2).subs(eqZ4).simplify()
T_w = T_w.subs(eqY2).subs(eqY4).simplify()
display(T_w)

Gx*(A + 2)/(-2*A*G + A*(G + Gx + Gy) + 2*G + 2*Gx + 2*Gy)

In [29]:
# 1. calculate the voltage across Gy
# Vy = V2 = Vin * T
# 2. From that calculate the Current through Gy
IGy = (T_w * Gy * Vin).simplify()
display(IGy)

Gx*Gy*Vin*(A + 2)/(-2*A*G + A*(G + Gx + Gy) + 2*G + 2*Gx + 2*Gy)

In [31]:
# 3. Simplify for A → ∞
IGys = sp.limit(IGy, A, sp.oo)
display(IGys)

-Gx*Gy*Vin/(G - Gx - Gy)

In [32]:
# What value do you have to choose for Gx such that the current through Gy does not depend on Gy anymore?
IGyss = sp.limit(IGys, Gx, G)
display(IGyss)

G*Vin

### Task b) Bandwidth

In [48]:
w1=sp.symbols('w1')
IGyA = IGy.subs(A,w1/s).subs(Gx,G).simplify()
display(niceT(IGyA,s))

(2*G*Gy*Vin*s + G*Gy*Vin*w1)/(Gy*w1 + 2*s*(2*G + Gy))

In [56]:
num,den = sp.fraction(IGyA)
wp = sp.solve(den, s)
wz = sp.solve(num, s)
display(-wp[0])
display(-wz[0])


Gy*w1/(4*G + 2*Gy)

w1/2

It is difficult to say what the exact bandwidth is, but if 2G is considerably larger than Gy (i.e., a big resistance is driven), then the pole dominates and the bandwidth is approximately

In [63]:
num,den=sp.fraction(-wp[0])
BW = num / sp.limit(den,Gy,0)
BW

Gy*w1/(4*G)

In [65]:
( BW / (2*sp.pi) ).subs(G,1).subs(Gy,1e-3).simplify()

0.000125*w1/pi